In [ ]:
from tqdm.notebook import tqdm

import catalog
import waveform

tqdm.pandas()
catalog.display_parameters()

In [ ]:
ESEC_avalanches = catalog.load("catalog/3_ESEC_avalanches_88.pickle")

ESEC_avalanches = ESEC_avalanches.reset_index(drop=True) ## To reset all avalanche indexes and end at 88 events
ESEC_avalanches['numero'] = range(88) ## To have a better manipulation of the events

catalog.open_plot(ESEC_avalanches, pos_number=0.11, xlim=100)

## Fetch available stations for each event

Get the inventory of waveforms per avalanche for a radius = 5° and put it in a new column

In [ ]:
ESEC_avalanches["inventory"] = ESEC_avalanches.progress_apply( 
    lambda row: waveform.download_inventory(row, maxradius=5, retries=10), axis=1
)

ESEC_avalanches.to_pickle("catalog/4_ESEC_avalanches_88.pickle")

## Download and save the waveforms

This cell download the waveform of all events in the catalog (if any) and save them into the waveforms directory in the data folder. If no data is available, then the event is skipped.

In [ ]:
for event in tqdm(ESEC_avalanches.itertuples(), total=len(ESEC_avalanches)):
    #if event.Index == 0 : ## Uncomment this line to download a specific event
        stream = waveform.download_stream(event, time_margins=500, print_error=True)
        if stream:
            stream.write(f"sismogrammes/{event.Index:03d}.pickle", format="PICKLE")